In [1]:
import pydicom
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib 
import os
import math
from totalsegmentator.python_api import totalsegmentator
%matplotlib inline

/home/et/miniconda3/envs/totalsegmentator/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
files = os.listdir('ct_dicom')

In [ ]:
for i in range(len(files)):
    #read the dicom examiantion and sort the slices based on the slice location
    def read_dicom(path):

        slices = [pydicom.dcmread(path + '/' + s, force=True) for s in os.listdir(path)]
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
        return slices

    dicom_image_serie = read_dicom('ct_dicom/' + files[i])
    dose_image_serie = read_dicom('dose_dicom/' + files[i] + '/aec_120kV_DoseDistribution')

    #convert the dicom image to a numpy array
    def dicom_to_array(slices):
            
        image = np.stack([s.pixel_array for s in slices])
        image = image.astype(np.int16)
        return image

    dicom_image_array = dicom_to_array(dicom_image_serie)
    dose_image_array = dicom_to_array(dose_image_serie)

    #converting pixel values to Hounsfield units
    def convert_to_hu(dicom_image_serie, dicom_image_array):
        intercept = dicom_image_serie[0].RescaleIntercept
        slope = dicom_image_serie[0].RescaleSlope
        hu_image = dicom_image_array * slope + intercept
        return hu_image
    
    
    def convert_to_dose(dose_image_array, mean_mA):
        dose_image_array = dose_image_array * ((21.4*mean_mA*0.5)/100000)
        return dose_image_array
    
    #extracting mA values
    mA = [s.XRayTubeCurrent for s in dicom_image_serie]
    mean_mA = np.mean(mA)

    #extracting slice location
    slice_location = [s.SliceLocation for s in dicom_image_serie]

    
    hu_image = convert_to_hu(dicom_image_serie, dicom_image_array)
    dose_image_array = convert_to_dose(dose_image_array, mean_mA)

    #creating organ masks

    totalsegmentator('ct_dicom/' + files[i], 'organ_masks/' + files[i])
    os.system('TotalSegmentator -i %s -o %s -ta body' % ('ct_dicom/' + files[i], 'organ_masks/' + files[i]))

    #WED calculation

    #voxel x and y dimensions
    x_dim = float(dicom_image_serie[0].PixelSpacing[0])
    y_dim = float(dicom_image_serie[0].PixelSpacing[1])

    body_mask = np.rot90(nib.load('organ_masks/' + files[i] + '/body.nii.gz').get_fdata())
    #transose the array to match the dimensions of the hu_image
    body_mask = np.transpose(body_mask, (2, 0, 1))
    body_mask[body_mask != 1] = np.nan

    wed_array = []

    for j in range(0, len(dicom_image_serie)):
        pix_number = np.count_nonzero(body_mask[j] == 1)
        area = pix_number*x_dim*y_dim
        croped = hu_image[j]*body_mask[j]
        mean_HU = np.nanmean(croped)
        wed = 2*(math.sqrt(((mean_HU/1000)+1)*(area/math.pi)))
        wed_array.append(wed)

    #training array creation

    def training_array(organ):
        mask = np.rot90(nib.load('organ_masks/' + files[i] + '/' + organ + '.nii.gz').get_fdata())
        mask = np.transpose(mask, (2, 0, 1))
        mask[mask != 1] = np.nan

        #retrieving the organ's HU values
        masked_organ = hu_image*mask

        #retieving dose values to organ's voxels
        dose_to_organ = dose_image_array*mask

        organ_training_array = np.empty((len(dicom_image_serie), 6), dtype=np.float32)
        idx = []
        for j in range(len(dicom_image_serie)):
            if np.all(np.isnan(masked_organ[j])):
                idx.append(j)
            else:
                organ_training_array[j][0] = np.nanmean(masked_organ[j])
                organ_training_array[j][1] = np.nanstd(masked_organ[j])
                organ_training_array[j][2] = float(mA[j])
                organ_training_array[j][3] = wed_array[j]
                organ_training_array[j][4] = float(slice_location[j])
                organ_training_array[j][5] = round(np.nanmean(dose_to_organ[j]), 1)

        clean_organ_training_array = np.delete(organ_training_array, idx, axis=0)

        #save the training array
        os.makedirs('training_arrays/' + organ, exist_ok=True)
        file_name = f'training_arrays/{organ}/{files[i]}_{organ}_training_array.npy'
        np.save(file_name, clean_organ_training_array)
        #return clean_organ_training_array
    
    
    organs_list = ['esophagus', 'aorta', 'pulmonary_artery', 'lung_lower_lobe_left', 'lung_lower_lobe_right',
                    'lung_upper_lobe_left', 'lung_upper_lobe_right', 'lung_middle_lobe_right', 
                    'heart_atrium_left', 'heart_atrium_right', 'heart_ventricle_left', 'heart_ventricle_right', 
                    'heart_myocardium', 'skin']

    for organ in organs_list:
        training_array(organ)

In [7]:
#append arrays

def append_arrays(organ):
    files = os.listdir('training_arrays/' + organ)
    appended_array = np.empty((0, 6), dtype=np.float32)
    for file in files:
        array = np.load('training_arrays/' + organ + '/' + file)
        appended_array = np.append(appended_array, array, axis=0)
    np.save('training_arrays/merged_training_arrays/' + organ + '_appended_array.npy', appended_array)

organs_list = ['esophagus', 'aorta', 'pulmonary_artery', 'lung_lower_lobe_left', 'lung_lower_lobe_right',
                    'lung_upper_lobe_left', 'lung_upper_lobe_right', 'lung_middle_lobe_right', 
                    'heart_atrium_left', 'heart_atrium_right', 'heart_ventricle_left', 'heart_ventricle_right', 
                    'heart_myocardium', 'skin']

for organ in organs_list:
    append_arrays(organ)